# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Задачи проекта**
1. Обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.
2. Построить модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Шаги проекта**
1. Проверка данных
2. Очищение и лемантизация комментариев
3. Разделение данных на обучающий и тестовый набор
4. Поиск лучшей модели по метрике F1
5. Проверка лучшей модели на тестовых данных

In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV 
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

import spacy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from tqdm.auto import tqdm
tqdm.pandas()

## Подготовка

### Загрузка и проверка данных

In [3]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
print('Дубликатов в данных:', data.duplicated().sum())

Дубликатов в данных: 0


In [25]:
print('Токсичных комментариев',round(100*data[data['toxic'] == 1]['toxic'].count()/len(data),2),'%')

Токсичных комментариев 10.16 %


### Очистка и лемантизация комментариев

In [6]:
def lemmatizer_spacy(text):        
    sent = []
    string = nlp(text)
    for word in string:
        sent.append(word.lemma_)
    return " ".join(sent)

In [7]:
lemmatizer = WordNetLemmatizer()
def lemmatizer_wnl(text):        
    sent = []
    string = nltk.word_tokenize(text)
    for word in string:
        sent.append(lemmatizer.lemmatize(word))
    return " ".join(sent)

In [8]:
def text_cleaner(string):
    string = re.sub(r"(?:\n|\r)", " ", string)
    string = re.sub(r"[^a-zA-Z ]+", "", string).strip()
    srting = string.lower()
    return srting

In [9]:
data['text'] = data['text'].progress_apply(text_cleaner)

  0%|          | 0/159292 [00:00<?, ?it/s]

**Лемантизация и обучение на всей выборке длится огромное время, возьмем подвыброрку в 10000 строк. Мощности юпитерхаб от яндекса не позволяют быстро проводить такие вычисления.**

In [10]:
# код для сэмплирования
#df = pd.DataFrame()
#df = data.sample(10000)
#df['lem'] = df.text.progress_apply(lemmatizer)

In [10]:
#data['lem'] = data['text'].progress_apply(lemmatizer_spacy)

In [11]:
#data['lem'] = data['text']

In [10]:
data['lem'] = data['text'].progress_apply(lemmatizer_wnl)

  0%|          | 0/159292 [00:00<?, ?it/s]

## Обучение

### Разделение данных

In [11]:
data_train, data_test = train_test_split(data, test_size=0.3, random_state=42)
# для сэмпла данных data_train, data_test = train_test_split(df, test_size=0.3, random_state=42)



### Создание корпуса слов и TFIDF

In [12]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

corpus = data_train['lem'].values
corpus_test = data_test['lem'].values

tfidf_train = count_tf_idf.fit_transform(corpus)
tfidf_test = count_tf_idf.transform(corpus_test)


### Логистическая регрессия

In [ ]:
%%time

#долгий поиск, лучше не запускать в юпитерхаб
pipe = Pipeline([
    (
    ('model', LogisticRegression(random_state=42, solver='saga', max_iter=200))
    )
    ])

param_distributions = [
        {

            'model': [LogisticRegression(random_state=42, solver='saga',max_iter=200,class_weight='balanced')],
            'model__penalty' : ['l1'],
            'model__C': list(range(1,15,1))
        }
]
grid_search = GridSearchCV(pipe, param_grid=param_distributions, 
                                       scoring='f1', cv=3, verbose=2, n_jobs=-1)

lr = grid_search.fit(tfidf_train,data_train.toxic)
print('Best parameters is:', grid_search.best_params_)
print('Best score is:', grid_search.best_score_)

Fitting 3 folds for each of 14 candidates, totalling 42 fits


### Случайный лес

In [29]:
%%time
pipe_2 = Pipeline([
    (
    ('model', RandomForestClassifier())
    )
    ])


param_distributions_rf = [
        {

            'model': [RandomForestClassifier()],
            'model__n_estimators': list(range(50,200,10)),
            'model__max_depth': list(range(2,10,1))
        }
]
randomized_search_2 = RandomizedSearchCV(pipe_2, param_distributions=param_distributions_rf, 
                                       scoring='f1', cv=3, verbose=2, n_jobs=-1)

rf = randomized_search_2.fit(tfidf_train[:50000],data_train.toxic[:50000])
print('Best parameters is:', randomized_search_2.best_params_)
print('Best score is:', randomized_search_2.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END model=RandomForestClassifier(), model__max_depth=2, model__n_estimators=150; total time=   0.7s
[CV] END model=RandomForestClassifier(), model__max_depth=2, model__n_estimators=150; total time=   0.7s
[CV] END model=RandomForestClassifier(), model__max_depth=2, model__n_estimators=150; total time=   0.7s
[CV] END model=RandomForestClassifier(), model__max_depth=3, model__n_estimators=60; total time=   0.4s
[CV] END model=RandomForestClassifier(), model__max_depth=3, model__n_estimators=60; total time=   0.4s
[CV] END model=RandomForestClassifier(), model__max_depth=3, model__n_estimators=60; total time=   0.4s
[CV] END model=RandomForestClassifier(), model__max_depth=9, model__n_estimators=150; total time=   2.1s
[CV] END model=RandomForestClassifier(), model__max_depth=9, model__n_estimators=150; total time=   2.1s
[CV] END model=RandomForestClassifier(), model__max_depth=9, model__n_estimators=150; total time=   2.

**Модель случайного леса совершенно не подходит для данной задачи. Будем использовать логистическую регрессию**

### Валидация лучшей модели

In [13]:
%%time
lr = LogisticRegression(random_state=42, C=4, penalty ='l2', solver='saga', class_weight='balanced',
                        max_iter=500, n_jobs=-1)
lr.fit(tfidf_train, data_train.toxic)

CPU times: user 53.3 s, sys: 37.6 ms, total: 53.4 s
Wall time: 53.4 s


LogisticRegression(C=4, class_weight='balanced', max_iter=500, n_jobs=-1,
                   random_state=42, solver='saga')

In [15]:
test_pred = lr.predict(tfidf_test)
f1_score(data_test.toxic, test_pred)

0.7609615384615385

## Выводы

Для предсказания тональности комментариев была использована модель логистической регрессии:
1. Подготовка данных занимает большую часть времени, данные были очищены от лишних символов и приведены к начальным формам
2. Использования части данных для обучения не позволяет достичь необходимой точности
3. Были найдены гиперпараметры для логистической регресии:C=4, penalty ='l2', solver='saga', class_weight='balanced', max_iter=500, n_jobs=-1. При этих параметрах метрика составила 0.76

